# Improvement Trial With 100% Separated Models

starting initial tests with a smaller number of trials just to see general performance

In [1]:
import gpu_rec
from constants import *
from copy import deepcopy
from random import shuffle
from game import Game
import numpy as np
import sys

Using TensorFlow backend.


In [2]:
#game is now using lag=1 instead of lag=2 for q-score
#maybe this will help guide it to higher point values in short term, 
#with less opportunities for probabilities straying
test_config = deepcopy(DEFAULT_GAME_CONFIG)
test_config['memory'] = 1
test_config['discount'] = 0.01

init_game = Game(n_players=5, q_lag=1, config=test_config)
#goal: eventually train players to not want to grab more tickets

players = init_game.players
#former will be shuffled
player_list = [p for p in players]
overall_player_wins = []
overall_player_scores = []
overall_winning_scores = []
overall_game_turns = []



In [3]:
temperatures = [0.6, 0.2, 0.1, 0.1, 0.05, 0.01, 0.01, 0., 0., 0., 0., 0., 0., 0.]
n_trials_per_cycle = 100
n_cycles = 3

for temp_index, temperature in enumerate(temperatures):
    print 'TEMPERATURE: %.03f' % temperature
    if temp_index==9:
        test_config['discount'] = 1
    elif temp_index==10:
        test_config['discount'] = 2
    elif temp_index==11:
        test_config['discount'] = 10
    elif temp_index==12:
        test_config['discount'] = 15
    elif temp_index==13:
        test_config['discount'] = 16
    test_config['temperature'] = temperature
    player_wins = [0 for _ in players]
    winning_scores = []
    player_scores = [[] for _ in players]
    game_turns = []
    for cycle in range(n_cycles):
        print 'in cycle %d' % cycle
        for i in range(n_trials_per_cycle):
            sys.stdout.write('.')
            shuffle(players)
            game = Game(pre_existing_players=players, config=test_config, separated_models=True)
            game.run(False)
            for i, player in enumerate(player_list):
                player_wins[i]+= player.win
                player_scores[i].append(player.total_points)
            winning_scores.append(game.winning_score)
            game_turns.append(game.turn)
        #don't train player 0, train player 1 partially, and train players 2 and 3 much more
        for i, player in enumerate(player_list):
            if i==1:
                player.ai.train_win()
                player.ai.train_q()
            elif i > 1:
                player.ai.train_win(20)
                player.ai.train_q(20)
    for player in player_list:
        player.ai.reset_history()
    overall_player_wins.append(player_wins)
    overall_player_scores.append(player_scores)
    overall_winning_scores.append(winning_scores)
    overall_game_turns.append(game_turns)
    #print out some summaries
    print 'WINNING SCORE AVERAGE: %.1f' % np.mean(winning_scores)
    print 'NUMBER OF TURNS AVERAGE: %.1f' % np.mean(game_turns)
    for i, player in enumerate(player_list):
        print '---PLAYER %d STATS---' % i
        print 'WIN AVERAGE: %.3f' % (np.mean(player_wins[i])/(float(n_trials_per_cycle*n_cycles)))
        print 'SCORE AVERAGE: %.1f' % np.mean(player_scores[i])
        

TEMPERATURE: 0.600
in cycle 0
....................................................................................................in cycle 1
....................................................................................................in cycle 2
....................................................................................................WINNING SCORE AVERAGE: 0.3
NUMBER OF TURNS AVERAGE: 267.0
---PLAYER 0 STATS---
WIN AVERAGE: 0.500
SCORE AVERAGE: -15.8
---PLAYER 1 STATS---
WIN AVERAGE: 0.187
SCORE AVERAGE: -24.1
---PLAYER 2 STATS---
WIN AVERAGE: 0.127
SCORE AVERAGE: -27.4
---PLAYER 3 STATS---
WIN AVERAGE: 0.020
SCORE AVERAGE: -37.5
---PLAYER 4 STATS---
WIN AVERAGE: 0.180
SCORE AVERAGE: -23.8
TEMPERATURE: 0.200
in cycle 0
....................................................................................................in cycle 1
....................................................................................................in cycle 2
.................................

ai.py:407: RuntimeWarning: invalid value encountered in divide
  return (selected)/np.sum(selected)
ai.py:379: RuntimeWarning: invalid value encountered in less
  return np.random.choice(len(new_probs), p=new_probs)


...................................................................................................in cycle 1
....................................................................................................in cycle 2
....................................................................................................WINNING SCORE AVERAGE: 5.4
NUMBER OF TURNS AVERAGE: 240.2
---PLAYER 0 STATS---
WIN AVERAGE: 0.220
SCORE AVERAGE: -24.4
---PLAYER 1 STATS---
WIN AVERAGE: 0.340
SCORE AVERAGE: -15.4
---PLAYER 2 STATS---
WIN AVERAGE: 0.227
SCORE AVERAGE: -21.3
---PLAYER 3 STATS---
WIN AVERAGE: 0.027
SCORE AVERAGE: -41.1
---PLAYER 4 STATS---
WIN AVERAGE: 0.210
SCORE AVERAGE: -24.8
TEMPERATURE: 0.010
in cycle 0
....................................................................................................in cycle 1
....................................................................................................in cycle 2
................................................................

its seems that temperature=0 produces the least desirable results, and increasing the discount factor doesn't help (since it's scaled to [0,1], anyway...)...also, why is player 0's score improving at all in spite of the fact that no training is done for that player???

In [5]:
print game.turn
print i
print cycle
print temp_index
for player in players:
    print player
    print len(player.determine_possible_tracks())

252
4
2
13
player with 0/3 completed/uncompleted tickets, 85 train cards, 33 cars, and 
		-15 total points
76
player with 0/7 completed/uncompleted tickets, 2 train cards, 13 cars, and 
		-53 total points
25
player with 0/7 completed/uncompleted tickets, 0 train cards, 0 cars, and 
		-24 total points
0
player with 1/7 completed/uncompleted tickets, 8 train cards, 4 cars, and 
		-37 total points
33
player with 1/7 completed/uncompleted tickets, 7 train cards, 6 cars, and 
		-20 total points
25


In [6]:
np.mean(overall_winning_scores,1)

array([  0.32666667,  -0.28      ,  -1.15      ,  -1.00666667,
        -1.28333333,   5.4       ,   5.49      ,  14.85      ,
        15.14333333,  10.67333333,  10.33      ,   6.65666667,
         9.78      ,  11.59      ])

In [7]:
np.mean(overall_player_scores, ( 2,))

array([[-15.82      , -24.11333333, -27.39      , -37.48333333,
        -23.76333333],
       [-12.18666667, -22.61333333, -27.21666667, -36.52333333, -25.31      ],
       [-15.46666667, -22.71666667, -26.86      , -38.20666667,
        -23.26666667],
       [ -9.62333333, -25.05333333, -26.81333333, -38.81666667, -25.36      ],
       [-12.10333333, -21.47      , -26.85333333, -38.73      , -25.42      ],
       [-24.44333333, -15.40666667, -21.31666667, -41.08333333,
        -24.80333333],
       [-26.00666667, -15.93666667, -20.31333333, -41.19666667,
        -24.19666667],
       [  9.80666667, -18.68666667, -27.99333333, -33.29333333,
        -26.57666667],
       [ 10.47      , -18.43666667, -28.4       , -32.22333333,
        -28.24333333],
       [  5.06666667, -18.75666667, -28.54333333, -32.5       ,
        -26.14666667],
       [  4.06      , -17.67333333, -28.1       , -32.37333333,
        -26.34333333],
       [  0.06666667, -17.55666667, -28.26333333, -33.53666667, -26

In [9]:
for i, player in enumerate(player_list):
    player.ai.save_models('ai_h5/p%d_test006_%%s.h5' % i)

In [10]:
for i, player in enumerate(player_list):
    print player.total_points
    print players[i].total_points

-15
-15
-37
-53
-24
-24
-53
-37
-20
-20
